In [1]:
# Basic DS stuff
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from urllib.request import urlopen
from thefuzz import fuzz
from fuzzywuzzy import process

Step 1: Load up our scraped dataset
---

In [2]:
games_df = pd.read_json('../data/raw/0 - Scraped Games DF.json', orient='records')

games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3154 entries, 0 to 3153
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   app_id                   3154 non-null   int64 
 1   title                    3154 non-null   object
 2   release_date             3154 non-null   object
 3   positive_review_percent  3128 non-null   object
 4   number_of_reviews        3128 non-null   object
 5   price                    2889 non-null   object
 6   game_page_link           3154 non-null   object
 7   tags                     3154 non-null   object
 8   developer                3154 non-null   object
 9   publisher                3154 non-null   object
 10  description              3154 non-null   object
 11  interface_languages      3154 non-null   object
 12  full_audio_languages     3154 non-null   object
 13  subtitles_languages      3154 non-null   object
 14  english                  3154 non-null  

In [3]:
games_df.head()

,app_id,title,release_date,positive_review_percent,number_of_reviews,price,game_page_link,tags,developer,publisher,...,german,french,spanish,brazilian,russian,italian,schinese,japanese,koreana,polish
0,1086940,Baldur's Gate 3,"Aug 3, 2023",95%,"259,208",$59.99,https://store.steampowered.com/app/1086940/Bal...,"[122,6426,1742,4747,21,4474,3843]",Larian Studios,Larian Studios,...,"(196,574)","(7,174)","(5,780)","(7,374)","(17,432)","(2,502)","(30,839)",(99),"(3,237)","(3,293)"
1,730,Counter-Strike: Global Offensive,"Aug 21, 2012",88%,"7,482,669",$14.99,https://store.steampowered.com/app/730/Counter...,"[1663,1774,3859,3878,19,5711,5055]","Valve, Hidden Path Entertainment",Valve,...,"(2,267,349)","(124,244)","(277,384)","(433,297)","(1,976,542)","(19,691)","(947,071)","(8,933)","(21,007)","(423,371)"
2,1888160,ARMORED CORE™ VI FIRES OF RUBICON™,"Aug 24, 2023",86%,"29,497",$59.99,https://store.steampowered.com/app/1888160/ARM...,"[4821,4747,19,1697,3993,4191,5752]",FromSoftware Inc.,FromSoftware Inc.,...,"(22,412)",(378),(391),(228),(609),(129),"(5,653)","(3,107)","(1,850)",(90)
3,1085660,Destiny 2,"Oct 1, 2019",81%,"562,194",NaN,https://store.steampowered.com/app/1085660/Des...,"[113,1695,1663,353880,1754,1685,1775]",Bungie,Bungie,...,"(341,920)","(10,917)","(17,154)","(17,647)","(39,239)","(3,607)","(87,572)",(875),"(13,355)","(6,208)"
4,1172470,Apex Legends™,"Nov 4, 2020",80%,"711,652",NaN,https://store.steampowered.com/app/1172470/Ape...,"[113,3859,176981,1774,1663,3839,1775]",Respawn Entertainment,Electronic Arts,...,"(365,479)","(15,472)","(23,684)","(13,900)","(85,332)","(3,208)","(102,293)","(7,259)","(14,868)","(11,794)"


Step 2: Standardize our data types
---

In [4]:
# The numbers in the comment count columns are strings
# with extra characters. Let's standardize them as ints.

# First, let's get a list of the columns that we need to process.
# That's all columns from the comments df and one column from the
# original game page scraped df.
%store -r top_10_languages
list_of_languages = top_10_languages.copy()
list_of_languages.append('english')

# Now, we iterate over all relevant columns, performing the operation
# that changes the value into a good int.
for index, row in games_df.iterrows() :
    for language in list_of_languages :
        # First we remove the parentheses.
        row[language] = row[language].strip('(|)')
        # Then we remove the commas.
        row[language] = row[language].replace(',', '')
        # Some cells stil have "failed" for this value, so we'll
        # convert those to NaNs for consistency.
        try :
            games_df.loc[index, language] = int(row[language])
        except :
            games_df.loc[index, language] = np.nan

In [5]:
# number_of_reviews has a similar problem. Should be ints.
# Games with no reviews can be passed over. We'll probably drop them later.
for index, row in games_df.iterrows() :
    try :
        row['number_of_reviews'] = row['number_of_reviews'].replace(',', '')
        games_df.loc[index, 'number_of_reviews'] = int(row['number_of_reviews'])
    except :
        pass

In [6]:
# Our positive review percentages should be ints as well.
games_df['positive_review_percent'] = games_df['positive_review_percent'].str.strip('%')

# I couldn't find an easy way to do this, so we're dumb-looping.
# Games with no positive review percent can be left alone - we don't need it, really.
for index, row in games_df.iterrows() :
    try :
        games_df.loc[index, 'positive_review_percent'] = int(row['positive_review_percent'])
    except :
        pass

In [7]:
# Prices should be ints. I want to avoid using floats since we might have to bin them
# and I don't want miniscule calc errors to mess me up, so we'll go with an int value
# that counts the number of pennies (so $5.99 = 599, etc.)
games_df['price'] = games_df['price'].str.strip('$')
games_df['price'] = games_df['price'].str.replace('.', '', regex=False)

for index, row in games_df.iterrows() :
    try :
        games_df.loc[index, 'price'] = int(row['price'])
    except :
        pass

In [8]:
# Now let's make release_date a datetime object.
# The only games that don't have release dates aren't released yet, so we can ignore them.
games_df['release_date'] = pd.to_datetime(games_df['release_date'], infer_datetime_format=True, errors='coerce')
games_df['release_date'] = games_df['release_date'].dt.strftime('%Y-%m-%d')

In [9]:
# For games that have multiple publishers, developers, etc, they're all stored as one string.
# If we want to use this data eventually, we should split it into lists.
# Some games don't have publishers or developers, and that's fine.
# Oddly, some of them have trailing spaces.
for index, row in games_df.iterrows() :
    row['developer'] = row['developer'].split(', ')
    # Since the above line changes the str into a list, we can .strip() each item in the list
    # with a comprehension.
    list_of_developers = [name.strip() for name in row['developer']]
    games_df.at[index, 'developer'] = list_of_developers

    # And again.
    row['publisher'] = row['publisher'].split(', ')
    list_of_publishers = [name.strip() for name in row['publisher']]
    games_df.at[index, 'publisher'] = list_of_publishers


In [10]:
# The tags are also just a string. We need them listed.
tags_list = []

for index, row in games_df.iterrows() :
    if row['tags'] != '' :
        tags_list = row['tags'].strip('[]').split(',')
        games_df.at[index, 'tags'] = tags_list

games_df['tags']

0            [122, 6426, 1742, 4747, 21, 4474, 3843]
1           [1663, 1774, 3859, 3878, 19, 5711, 5055]
2           [4821, 4747, 19, 1697, 3993, 4191, 5752]
3        [113, 1695, 1663, 353880, 1754, 1685, 1775]
4        [113, 3859, 176981, 1774, 1663, 3839, 1775]
                            ...                     
3149            [113, 128, 1741, 1754, 1695, 122, 9]
3150       [4736, 1743, 597, 6815, 4085, 7368, 4252]
3151            [122, 1646, 4231, 19, 21, 492, 1685]
3152               [9, 597, 17389, 1770, 1666, 3859]
3153    [493, 4328, 3834, 220585, 1643, 9157, 12472]
Name: tags, Length: 3154, dtype: object

In [11]:
games_df.columns

Index(['app_id', 'title', 'release_date', 'positive_review_percent',
       'number_of_reviews', 'price', 'game_page_link', 'tags', 'developer',
       'publisher', 'description', 'interface_languages',
       'full_audio_languages', 'subtitles_languages', 'english', 'german',
       'french', 'spanish', 'brazilian', 'russian', 'italian', 'schinese',
       'japanese', 'koreana', 'polish'],
      dtype='object')

In [12]:
# Same problem for our languages types columns (interface, audio, subitles).

for index, row in games_df.iterrows() :
    if row['interface_languages'] != '' :
        languages_list = row['interface_languages'].strip('[]').split(', ')
        games_df.at[index, 'interface_languages'] = languages_list
    if row['full_audio_languages'] != '' :
        languages_list = row['full_audio_languages'].strip('[]').split(', ')
        games_df.at[index, 'full_audio_languages'] = languages_list
    if row['subtitles_languages'] != '' :
        languages_list = row['subtitles_languages'].strip('[]').split(', ')
        games_df.at[index, 'subtitles_languages'] = languages_list

# Check.
print(type(games_df.loc[0, 'interface_languages']))

<class 'list'>


In [13]:
# For future reference, let's create a dictionary of tags codes & their meanings.
# We can get that from the search page.
# Keys will be the codes. Values will be the names.
url = 'https://store.steampowered.com/search'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

# The relevant data is in this one code block.
code_block = soup.find('div', id="TagFilter_Container")

# Create the empty dict.
tags_dict = {}

# Iterate over all 400+ tags described in the code block.
for listing in code_block.find_all('div', class_='tab_filter_control_row') :
    tag_code = listing.get('data-value')
    tag_name = listing.get('data-loc')
    tags_dict[tag_code] = tag_name

# We'll probably need it later, so let's save it a couple ways.
# Weirdly, it's quicker and easier to do this via a DF.
tags_dict_df = pd.DataFrame.from_dict(tags_dict, orient='index')
tags_dict_df.to_csv('../data/interim/Tags Dictionary DF.csv')
%store tags_dict

Stored 'tags_dict' (dict)


Implement indicator variables
---

In [14]:
# For language_types, implement pipes (???)

Step 3: Sanity checks
---

In [15]:
# Since our tags column is very important to us, let's make sure
# that the entries are reasonably comparable.

# In order to count the number of tags in each game's tags list,
# let's turn that column into a list of lists (by way of a series)
# so that we can use a list comprehension to get a list of len()s.
series_of_tags_column_values = games_df['tags']
list_of_tags_column_values = series_of_tags_column_values.tolist()
tags_values_lengths = [len(x) for x in list_of_tags_column_values]

# Now we organize the counts into a dictionary, where the keys are
# the lengths and the values are the frequencies of those lengths.
dict_of_lenghts = {}

for length in tags_values_lengths :
    if length in dict_of_lenghts :
        dict_of_lenghts[length] += 1
    else :
        dict_of_lenghts[length] = 1

print(dict_of_lenghts)

# Looks not too bad.

{7: 3140, 6: 5, 4: 4, 5: 4, 2: 1}


In [16]:
# Let's make sure our language-specific comment counts are possible.

# We know that the sum of our comment counts SHOULD be LESS than the
# number of total comments, because we aren't considering every language,
# just the top 10 loc languages + English. But we should make sure.

# First, we make a series of the sums of all the comment counts for
# all of our considered languages.
languages_comments_sums = games_df.loc[:, list_of_languages].sum(axis=1)

# Now, we see if they're ever greater than the total comment count.
error_rows = games_df[games_df['number_of_reviews'] < languages_comments_sums]

# How did we do?
error_rows.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3111 entries, 0 to 3153
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   app_id                   3111 non-null   int64 
 1   title                    3111 non-null   object
 2   release_date             3103 non-null   object
 3   positive_review_percent  3111 non-null   object
 4   number_of_reviews        3111 non-null   object
 5   price                    2851 non-null   object
 6   game_page_link           3111 non-null   object
 7   tags                     3111 non-null   object
 8   developer                3111 non-null   object
 9   publisher                3111 non-null   object
 10  description              3111 non-null   object
 11  interface_languages      3111 non-null   object
 12  full_audio_languages     3111 non-null   object
 13  subtitles_languages      3111 non-null   object
 14  english                  3111 non-null  

In [17]:
# Whoa! That's almost all of them! What gives? Let's see how much we're off by...
comments_differences = languages_comments_sums - games_df['number_of_reviews']

comments_differences_df = pd.DataFrame({'Displayed Total': games_df['number_of_reviews'], 'Calculated Total':languages_comments_sums, 'Difference':comments_differences})

comments_differences_df

,Displayed Total,Calculated Total,Difference
0,259208,460259.0,201051.0
1,7482669,8560724.0,1078055.0
2,29497,56766.0,27269.0
3,562194,862735.0,300541.0
4,711652,991284.0,279632.0
...,...,...,...
3149,9327,12552.0,3225.0
3150,117,212.0,95.0
3151,6885,12317.0,5432.0
3152,704,1628.0,924.0


In [18]:
# Good lord. Seems like we made some bad assumptions early on! The displayed total
# and calculated totals are off by a whole lot. In our case, since we're mostly
# concerned with the ratios, let's not worry about the displayed total - let's
# only worry about the number of comments we DO have.

# So, let's make that languages_comments_sum Series a column of our df, and just
# drop the original number_of_reveiws.

# And let's make it an int along the way, for good measure.
languages_comments_sums = languages_comments_sums.astype(int)
games_df.drop('number_of_reviews', axis=1, inplace=True)
games_df['total_comments'] = languages_comments_sums


In [19]:
# # Check for string similarity in all verbal cols.
# # In developer and publisher, this could likely find typos.

# # First, let's create sets of all the extant developer and publisher names
# # as currently typed.

# set_of_developer_names = set()

# for developers_list in games_df['developer'] :
#     for developer in developers_list :
#         set_of_developer_names.add(developer)

# set_of_publisher_names = set()

# for publishers_list in games_df['publisher'] :
#     for publisher in publishers_list :
#         set_of_publisher_names.add(publisher)

# # Now, let's see if any items in that list are super similar to each other.
# # This could possibly the the result of a typo.
# # Let's first find the MOST similar pair in each set.
# # If these pairs are clearly not typos, then it's very likely that there are
# # no typos at all.

# list_of_developer_similarities = []
# listing = []

# for developer in set_of_developer_names :
#     # Create a set of names to test this name against.
#     # We want to avoid testing the name against itself,
#     # since that will return a maximum similarity score
#     # and make our lives harder.
#     # So, we'll create a new set, then drop this name
#     # from the set before doing the comparisons.
#     testing_set = set_of_developer_names.copy()
#     testing_set.remove(developer)
#     highest = process.extract(developer, testing_set, limit=1)
#     # We need to retain both of the tested strings, while .extract only
#     # returns the second one. We'll have to group them manually.
#     # Let's also arbitrarily set 90 as the cutoff for similarity.
#     if highest[0][1] >= 90 :
#         listing = [developer, highest[0][0], highest[0][1]]
#         list_of_developer_similarities.append(listing)


# # Then we do it all again for publishers.
# list_of_publisher_similarities = []

# for publisher in set_of_publisher_names :
#     testing_set = set_of_publisher_names.copy()
#     testing_set.remove(publisher)
#     highest = process.extract(publisher, testing_set, limit=1)
#     if highest[0][1] >= 90 :
#         listing = [publisher, highest[0][0], highest[0][1]]
#         list_of_publisher_similarities.append(listing)


# # Now we sort the lists by the similarity score.
# list_of_developer_similarities.sort(key=lambda x: x[2], reverse=True)
# list_of_publisher_similarities.sort(key=lambda x: x[2], reverse=True)


# # Let's take a look at some of them... and also get a feel for
# # how long these lists are (and therefore how many of these names
# # are truly similar).
# print('Similar developer names:')
# print(list_of_developer_similarities[0:10])
# print('Total similarity scores over 90: '+str(len(list_of_developer_similarities)))
# print('')
# print('Similar publisher names:')
# print(list_of_publisher_similarities[0:10])
# print('Total similarity scores over 90: '+str(len(list_of_publisher_similarities)))

# # Well, what I've learned from this is that the names of developers and publishers
# # would be a LOT of work to fix. Since we don't need them for now (they aren't part
# # of our key analysis), we can just leave them as-is and clean them later if we need
# # them.

Step 4: Determine completeness
---

In [20]:
# While I want to keep all the data around just in case, this is a great
# point for us to determine whether some rows may not be useful for us.

# Our main label is the comment counts per language, so we can safely drop
# all rows that have no language-specific comment counts at all.

# Let's make a list of nan values 10 items long, to help us check the 
# 10 languages columns... if they're all nan, we can drop that game.
fail_list = [np.nan] * 10
indexes_to_drop = []

for index, row in games_df.iterrows() :
    test_list = row[14:24].tolist()
    if test_list == fail_list :
        indexes_to_drop.append(index)

print(len(indexes_to_drop))


25


In [21]:
# Only 25! Not bad. Not worth fussing over. Let's just throw them out
# directly.
games_df = games_df.drop(indexes_to_drop)

In [22]:
# Our main feature is the tags, so we should make sure that all rows have them.
indexes_to_drop = []

for index, row in games_df.iterrows() :
    if len(row['tags']) == 0 :
        indexes_to_drop.append(index)

print(len(indexes_to_drop))
# Excellent! Looks like we're in the clear there.

0


Step 5: A bit more standardization
---

In [23]:
# I've noticed that the way languages are written in the langauge types columns is different
# from the way they're written in the languages columns. We'll need them to match up if we
# want to do a comparative analysis.

# The problems are threefold.
# First, the capitalization is different.
# Second, the language types columns split up some languages (for example, Spanish is split
#   into "Spanish - Spain" and "Spanish - Latin America").
# Third, some are just spelled differently, like "schinese" and "Simplified Chinese".

# First, let's just lowcap everything. Turn down the volume a bit.
for index, row in games_df.iterrows() :
    dummy_list = []
    for item in row['interface_languages'] :
        item = item.lower()
        # Since we're already here, let's standardize the spellings.
        for language in list_of_languages :
            if language in item :
                item = language
        # And it looks like there's only one special case we need to handle specifically.
        if item == "'simplified chinese'" :
            item = 'schinese'
        # Add the results to the list...
        dummy_list.append(item)
    # ... and then make the list the new entry in the DF.
    games_df.at[index, 'interface_languages'] = dummy_list

    # Now we do the same for the other two langauge type columns...
    dummy_list = []
    for item in row['full_audio_languages'] :
        item = item.lower()
        # Since we're already here, let's standardize the spellings.
        for language in list_of_languages :
            if language in item :
                item = language
        # And it looks like there's only one special case we need to handle specifically.
        if item == "'simplified chinese'" :
            item = 'schinese'
        # Add the results to the list...
        dummy_list.append(item)
    # ... and then make the list the new entry in the DF.
    games_df.at[index, 'full_audio_languages'] = dummy_list

    dummy_list = []
    for item in row['subtitles_languages'] :
        item = item.lower()
        # Since we're already here, let's standardize the spellings.
        for language in list_of_languages :
            if language in item :
                item = language
        # And it looks like there's only two special cases we need to handle specifically.
        if item == "'simplified chinese'" :
            item = 'schinese'
        if 'portugese' in item :
            item = 'brazilian'
        # Add the results to the list...
        dummy_list.append(item)
    # ... and then make the list the new entry in the DF.
    games_df.at[index, 'subtitles_languages'] = dummy_list

Step 6: Wrangle a couple more key columns
---

In [24]:
# Let's look at how our prices are distributed...
list_of_prices = games_df['price']
counts = list_of_prices.value_counts()
print(counts[counts > 50])
print(counts[counts > 50].sum())

1999    686
2999    359
1499    270
2499    262
3999    242
999     215
5999    156
4999     91
499      79
3499     59
Name: price, dtype: int64
2419


In [51]:
# We can see that almost 5/6 of our games fall into just a
# few price points! That's significant to know. Let's use
# these as bins, then "below that" and "above that" as
# separate bins, for a total of 13 bins.

# I'm not 100% confident in the efficacy of this method,
# so let's preserve the original price data in the DF.
# Later, when modeling, we can see if there's a reason
# to use one or the other.

ranges = [0, 700, 1200, 1700, 2200, 2700, 3200, 3700, 4200, 4700, 5200, 5700, 6500, 7500, np.inf]
range_names = ['under 10', '999', '1499', '1999', '2499', '2999', '3499', '3999', '4499', '4999', '5499', '5999', '6999', 'over 80']
games_df['price_category'] = pd.cut(games_df['price'], bins=ranges, labels=range_names)

In [52]:
list_of_pairs = []

for i in range(len(ranges)) :
    list_of_pairs.append([ranges[i-1], range_names[i-1]])

for item in list_of_pairs :
    print(item)

[inf, 'lll']
[0, 'cat']
[700, 'dog']
[1200, 'house']
[1700, 'aaa']
[2200, 'bbb']
[2700, 'ccc']
[3200, 'ddd']
[3700, 'eee']
[4200, 'fff']
[4700, 'ggg']
[5200, 'hhh']
[5700, 'iii']
[6500, 'jjj']
[7500, 'lll']


In [53]:
games_df.loc[10:20, ['price', 'price_category']]


,price,price_category
10,1999,house
11,14582,lll
12,NaN,NaN
13,NaN,NaN
14,3999,cat
15,NaN,NaN
16,5999,cat
17,3999,dog
18,4999,ggg
19,3499,ddd


In [26]:
# We have all the info we need now, but it's not quite in a form
# that we can use. The total number of comments per language is
# meaningless - we need to know the average proportion of comments in
# that language, so we can tell if any individual game (and therefore
# any specific combination of tags) generates more engagement in that
# market than others do.

# Unfortunately, there's one other thing that probably impacts the
# proportion of comments much more than the game itself - whether or
# not the game has been localized into that language at all. So we can
# control for that, making multiple constants...

# To find the average, we'll need the total amount of comments in all langs
# for all games.
total_comments = games_df.total_comments.sum()

# Now we can programmatically calculate the percentage of ALL comments
# that each language occupies.
language_averages_agnostic = {}

# Iterate over the languages, summing all comments in that language to find
# the overall average.
for language in list_of_languages :
    total_comments_l = games_df[language].sum()
    language_average = total_comments_l / total_comments
    language_averages_agnostic[language] = language_average

print(language_averages_agnostic)


{'german': 0.35208743896781586, 'french': 0.016248295029496828, 'spanish': 0.031898891788261836, 'brazilian': 0.030559324275612868, 'russian': 0.08999471386938188, 'italian': 0.004338109585673992, 'schinese': 0.11067257090131, 'japanese': 0.0029063012018170505, 'koreana': 0.014918114274571112, 'polish': 0.01865944683629249, 'english': 0.3277167932697661}


In [27]:
# Above is the calculation if we ignore whether or not the game is even in that
# language. Let's see if it's much different when we take into account text, voice,
# and subtitle translation, or any of the three.

# Let's subset for these 4 situations, then run the same calculations again.

language_averages_any = {}
language_averages_interface = {}
language_averages_audio = {}
language_averages_subtitles = {}


# Do it for all languages, all situations.
for language in list_of_languages :

    # First, the condition of the target language being localized in any way.
    running_language_comments = 0
    running_total_comments = 0

    for index, row in games_df.iterrows() :
        if language in row['interface_languages'] or language in row['full_audio_languages'] or language in row['subtitles_languages'] :
            if not pd.isna(row[language]) :
                running_language_comments += int(row[language])
            if not pd.isna(row['total_comments']) :
                running_total_comments += row['total_comments']
    if running_total_comments != 0 :
        language_averages_any[language] = running_language_comments / running_total_comments


    # Now, the condition of the target language being an interface language.
    running_total_comments = 0
    running_language_comments = 0

    for index, row in games_df.iterrows() :
        if language in row['interface_languages'] :
            if not pd.isna(row[language]) :
                running_language_comments += row[language]
            if not pd.isna(row['total_comments']) :
                running_total_comments += row['total_comments']
        if running_total_comments != 0 :
            language_averages_interface[language] = running_language_comments / running_total_comments


    # Now, the condition of the target language being an audio langauge.
    running_total_comments = 0
    running_language_comments = 0

    for index, row in games_df.iterrows() :
        if language in row['full_audio_languages'] :
            if not pd.isna(row[language]) :
                running_language_comments += row[language]
            if not pd.isna(row['total_comments']) :
                running_total_comments += row['total_comments']
    if running_total_comments != 0 :
        language_averages_audio[language] = running_language_comments / running_total_comments


    # Now, the condition of the target language being a subtitle langauge.
    running_total_comments = 0
    running_language_comments = 0

    for index, row in games_df.iterrows() :
        if language in row['subtitles_languages'] :
            if not pd.isna(row[language]) :
                running_language_comments += row[language]
            if not pd.isna(row['total_comments']) :
                running_total_comments += row['total_comments']
    if running_total_comments != 0 :
        language_averages_subtitles[language] = running_language_comments / running_total_comments



In [28]:
print(language_averages_agnostic)
print(language_averages_any)
print(language_averages_interface)
print(language_averages_audio)
print(language_averages_subtitles)

{'german': 0.35208743896781586, 'french': 0.016248295029496828, 'spanish': 0.031898891788261836, 'brazilian': 0.030559324275612868, 'russian': 0.08999471386938188, 'italian': 0.004338109585673992, 'schinese': 0.11067257090131, 'japanese': 0.0029063012018170505, 'koreana': 0.014918114274571112, 'polish': 0.01865944683629249, 'english': 0.3277167932697661}
{'german': 0.3495844163881065, 'french': 0.01691389340774637, 'spanish': 0.032742336254021884, 'russian': 0.09868213062545923, 'italian': 0.004529224165326765, 'schinese': 0.12948826487112614, 'japanese': 0.0032912493805137715, 'polish': 0.021359508711750574, 'english': 0.3277167932697661}
{'german': 0.34954993078233815, 'french': 0.016917827827072917, 'spanish': 0.032742605026873206, 'russian': 0.0986965105605507, 'italian': 0.004529396356586199, 'schinese': 0.1295657798506253, 'japanese': 0.0032973418726349292, 'polish': 0.021331320968752874, 'english': 0.3277167932697661}
{'german': 0.3560838498289804, 'french': 0.01754508367512497,

In [29]:
# I'm not sure if this is "wrangling"  or "feature engineering," but I might as well
# end this section with a full dataset...

# Let's create a column with the DIFFERENCES between the %s of comments in a language
# on a certain game and the average number of comments in that language on all games.
# A positive number here will indicate above-average interest within this language
# group for this game, and a negative number will indicate below-average interest.

# We'll make 5 such columns, one for each of the "language_averages" metrics. Later,
# during EDA, we can see which (if any) of them yield a clearer result.

# Since we have what we need already, let's just dig right in to creating columns.
# Let's make one column for each metric, and store all language values in a dict
# for each game in that column.

holding_dict = {}

games_df['comment_ratios'] = ''
games_df['comment_diff_agnostic'] = ''
games_df['comment_diff_any'] = ''
games_df['comment_diff_interface'] = ''
games_df['comment_diff_audio'] = ''
games_df['comment_diff_subtitles'] = ''


# First we need to create the dict of the ratio of comments for that specific
# language/game pair.
for index, row in games_df.iterrows() :

    for language in list_of_languages :
        score = np.nan
        score = row[language] / row['total_comments']
        holding_dict[language] = score
    games_df.at[index, 'comment_ratios'] = holding_dict 
    holding_dict={}

# Now that that column is filled in, we can populate the rest of them with calculations.
for index, row in games_df.iterrows() :

    # Start with the agnostic metric.
    for language in list_of_languages :
        score = np.nan
        score = row['comment_ratios'][language] - language_averages_agnostic[language]
        holding_dict[language] = score
    games_df.at[index, 'comment_diff_agnostic'] = holding_dict 
    holding_dict={}

    # Now we do the 'any' metric
    for language in list_of_languages :
        score = np.nan
        if language in row['interface_languages'] or \
                language in row['full_audio_languages'] or \
                language in row['subtitles_languages'] :
            score = row['comment_ratios'][language] - language_averages_any[language]
        holding_dict[language] = score
    games_df.at[index, 'comment_diff_any'] = holding_dict
    holding_dict={}

    # "interface_languages" metric.
    for language in list_of_languages :
        score = np.nan
        if language in row['interface_languages'] :
            score = row['comment_ratios'][language] - language_averages_interface[language]
        holding_dict[language] = score
    games_df.at[index, 'comment_diff_interface'] = holding_dict
    holding_dict={}

    # "full_audio" metric.
    for language in list_of_languages :
        score = np.nan
        if language in row['full_audio_languages'] :
            score = row['comment_ratios'][language] - language_averages_audio[language]
        holding_dict[language] = score
    games_df.at[index, 'comment_diff_audio'] = holding_dict
    holding_dict={}

    # Last but not least, 'subtitles.'
    for language in list_of_languages :
        score = np.nan
        if language in row['subtitles_languages'] :
            score = row['comment_ratios'][language] - language_averages_subtitles[language]
        holding_dict[language] = score
    games_df.at[index, 'comment_diff_subtitles'] = holding_dict

In [30]:
# Now that all our ratios are in dictionary form, it feels kind of silly for the
# comment counts to have their own columns. Let's dictify that one as well.

games_df['language_comment_counts'] = ''

holding_dict = {}

for index, row in games_df.iterrows() :
    for language in list_of_languages :
        holding_dict[language] = row[language]
    games_df.at[index, 'language_comment_counts'] = holding_dict


Step 6: Pare down the columns
---

In [32]:
# Not all of our columns are useful for analysis, so let's remove some of them to make things
# easier to look at. I'll keep a version of the full dataframe saved as well, just in case
# we later decide that we needed some of that info. Then we can just add it back in using
# app_id as the key.

games_df.to_csv('../data/interim/0.5 - Games DF - Wrangled, Extra Columns.csv')

In [33]:
# Take a look and see what we need...
games_df.columns

Index(['app_id', 'title', 'release_date', 'positive_review_percent', 'price',
       'game_page_link', 'tags', 'developer', 'publisher', 'description',
       'interface_languages', 'full_audio_languages', 'subtitles_languages',
       'english', 'german', 'french', 'spanish', 'brazilian', 'russian',
       'italian', 'schinese', 'japanese', 'koreana', 'polish',
       'total_comments', 'price_category', 'comment_ratios',
       'comment_diff_agnostic', 'comment_diff_any', 'comment_diff_interface',
       'comment_diff_audio', 'comment_diff_subtitles',
       'language_comment_counts'],
      dtype='object')

In [34]:
# Now we'll make a list of columns we really don't need and take them out.

columns_to_drop = ['title', 'positive_review_percent', 'game_page_link', 'developer', 'publisher', \
                   'description']

# We don't need the language count columns any more, either.
columns_to_drop += list_of_languages

games_df.drop(columns_to_drop, axis=1, inplace=True)

games_df.columns

Index(['app_id', 'release_date', 'price', 'tags', 'interface_languages',
       'full_audio_languages', 'subtitles_languages', 'total_comments',
       'price_category', 'comment_ratios', 'comment_diff_agnostic',
       'comment_diff_any', 'comment_diff_interface', 'comment_diff_audio',
       'comment_diff_subtitles', 'language_comment_counts'],
      dtype='object')

In [35]:
# The columns don't seem to be in a logical order. Let's fix that manually.
new_column_order = ['app_id', 'release_date', 'price', 'price_category', 'tags', 'interface_languages', \
                    'full_audio_languages', 'subtitles_languages', 'total_comments', 'language_comment_counts', \
                    'comment_ratios', 'comment_diff_agnostic', 'comment_diff_any', 'comment_diff_interface', \
                    'comment_diff_audio', 'comment_diff_subtitles']

games_df = games_df.reindex(columns=new_column_order)

games_df.columns

Index(['app_id', 'release_date', 'price', 'price_category', 'tags',
       'interface_languages', 'full_audio_languages', 'subtitles_languages',
       'total_comments', 'language_comment_counts', 'comment_ratios',
       'comment_diff_agnostic', 'comment_diff_any', 'comment_diff_interface',
       'comment_diff_audio', 'comment_diff_subtitles'],
      dtype='object')

Step 7: Save and quit
---

In [36]:
games_df.to_json('../data/processed/1 - Games DF - Wrangled.json')